In [742]:
import pandas as pd
import bqplot
import numpy as np
import ipywidgets
from datetime import datetime
import bqplot
from bqplot import pyplot as plt



In [743]:
df = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv",
                 na_values = {'Year Acquired': 0, 
                                     'Year Constructed': 0, 
                                     'Square Footage': 0})


In [744]:
df['Year Acquired'] = pd.to_datetime(df['Year Acquired'], format='%Y')
dfheatmap = pd.pivot_table(df,  index=["Agency Name"],
                                values=["Square Footage"],
                                columns=["County"],
                                aggfunc=np.mean)

In [745]:
col_sc = bqplot.ColorScale(scheme='inferno')
x_sc = bqplot.OrdinalScale() 
y_sc = bqplot.OrdinalScale()

In [746]:
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
x_ax = bqplot.Axis(scale=x_sc, label='County', label_offset='100px',tick_rotate=60,tick_style={'font-size':'7px'})
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Agency Name', tick_style={'font-size':'5px'})

In [747]:
anames = dfheatmap.columns.levels[1].to_list() 

In [748]:
heat_map = bqplot.GridHeatMap(color = np.log10(dfheatmap.values),
                              row = dfheatmap.index, 
                              column = anames,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

In [749]:
fig.layout.min_width='500px'
figures = ipywidgets.HBox([fig])
fig = bqplot.Figure(marks=[heat_map], axes=[col_ax,x_ax,y_ax])
myDashboard = ipywidgets.VBox([fig])
myDashboard


In [750]:
x_scl = bqplot.DateScale()
y_scl = bqplot.LinearScale()

ax_xcl = bqplot.Axis(label='Date', scale=x_scl)
ax_ycl = bqplot.Axis(label='Square footage in year acquired', scale=y_scl, 
                    orientation='vertical', side='left')

In [751]:
i,j = 0,0
mask = (df['Agency Name'] == dfheatmap.index[i]) & \
       (df['County'] == dfheatmap.columns.levels[1][j])
subset = df[mask]


In [752]:
group = subset.groupby("Year Acquired")["Square Footage"].sum()
years = group.index
sqft = group.values

In [753]:
bar = bqplot.Bars(x = years, 
                  y = sqft,
                  scales = {'x': x_scl,
                            'y': y_scl},
                 color_mode="auto",
                 colors=["fuchsia", "maroon", "purple", "red", "yellow"])

fig_bar = bqplot.Figure(marks = [bar], axes = [ax_ycl, ax_xcl])
fig_bar

Figure(axes=[Axis(label='Square footage in year acquired', orientation='vertical', scale=LinearScale(), side='…

In [754]:
mySelectedLabel = ipywidgets.Label()
def get_data_value(change):
    i,j = change['owner'].selected[0]
    mask = (df['Agency Name'] == dfheatmap.index[i]) & \
            (df['County'] == dfheatmap.columns.levels[1][j])
    subset = df[mask]
    
    group = subset.groupby("Year Acquired")["Square Footage"].sum()
    years = group.index
    sqft = group.values
    bar.x = years
    bar.y = sqft
heat_map.observe(get_data_value, 'selected')
fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax],
                    fig_margin = dict(top=50, bottom=150, left=50, right=50))
fig.layout.align_items
fig.layout.min_width='700px'
fig.layout.min_height='600px'
fig_bar.layout.min_width='250px'
ipywidgets.HBox([fig,fig_bar])
